# 🍏 Observability & Tracing Demo with `azure-ai-projects` and `azure-ai-inference` 🍎

Bienvenido a este cuaderno temático de **Health & Fitness**, donde exploraremos cómo configurar **observability** y **tracing** para:

1. **Basic LLM calls** usando un `AIProjectClient`.
2. **Multi-step** interacciones usando un **Agent** (como un Health Resource Agent).
3. Enviar esos **traces** a **Azure Monitor** (Application Insights) para que puedas visualizarlos en **Azure AI Foundry**.

> **Disclaimer**: Esta es una demostración divertida de AI y observability! Cualquier referencia a rutinas de ejercicios, dietas o regímenes de salud en el código o en los prompts son únicamente con fines **educational**. Siempre consulta a un profesional para obtener consejos de salud.

## Contents
1. **Initialization**: Configuración del entorno, creación de clientes.
2. **Basic LLM Call**: Demostración rápida de cómo obtener completions de modelos.
3. **Connections**: Listado de conexiones del proyecto.
4. **Observability & Tracing**
   - **Console / Local** tracing
   - **Prompty / Aspire**: envío de traces a un **OTLP endpoint** local
   - **Azure Monitor** tracing: conexión a Application Insights
   - **Verifying** de tus traces en Azure AI Foundry
5. **Agent-based Example**:
   - Creación de un simple "Health Resource Agent" haciendo referencia a documentos de ejemplo.
   - Conversación de múltiples turnos con tracing.
   - Cleanup.

<img src="./seq-diagrams/1-observability.png" width="50%"/>

## 1. Initialization & Setup
**Prerequisites**:
- Un archivo `.env` que contenga `PROJECT_CONNECTION_STRING` (y opcionalmente `MODEL_DEPLOYMENT_NAME`).
- Roles/permissions en Azure AI Foundry que te permiten hacer inference & agent creation.
- Un entorno local con paquetes `azure-ai-projects`, `azure-ai-inference`, `opentelemetry` instalados.

**What we do**:
- Cargar variables de entorno.
- Inicializar `AIProjectClient`.
- Verificar que podemos comunicarnos con un modelo (como `gpt-4o`).

In [1]:
import os
import sys
import time
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, CompletionsFinishReason

# Load environment variables
notebook_path = Path().absolute()
env_path = notebook_path.parent.parent / ".env"  # Adjust path as needed
load_dotenv(env_path, override=True)

connection_string = os.environ.get("PROJECT_CONNECTION_STRING")
if not connection_string:
    raise ValueError("🚨 PROJECT_CONNECTION_STRING not set in .env.")

# Initialize AIProjectClient
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(), conn_str=connection_string
    )
    print("✅ Successfully created AIProjectClient!")
except Exception as e:
    print(f"❌ Error creating AIProjectClient: {e}")

✅ Successfully created AIProjectClient!


## 2. Basic LLM Call
Realizaremos una solicitud rápida de chat completion para confirmar que todo esté funcionando. Haremos una pregunta simple: "¿Cuántos pies hay en una milla?"

In [2]:
try:
    # Create a ChatCompletions client
    inference_client = project_client.inference.get_chat_completions_client()
    # Default to "gpt-4o" if no env var is set
    model_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

    user_question = "¿Cuántos pies hay en una milla?"
    response = inference_client.complete(
        model=model_name, messages=[UserMessage(content=user_question)]
    )
    print("\n💡Response:")
    print(response.choices[0].message.content)
    print("\nFinish reason:", response.choices[0].finish_reason)

except Exception as e:
    print("❌ Could not complete the chat request:", e)


💡Response:
Hay 5280 pies en una milla.

Finish reason: CompletionsFinishReason.STOPPED


## 3. List & Inspect Connections
Mira las **connections** que tiene tu proyecto: estas pueden ser Azure OpenAI u otros adjuntos de recursos. Solo las listaremos aquí para demostración.

In [3]:
from azure.ai.projects.models import ConnectionType

all_conns = project_client.connections.list()
print(f"🔎 Found {len(all_conns)} total connections.")
for idx, c in enumerate(all_conns):
    print(
        f"{idx+1}) Name: {c.name}, Type: {c.connection_type}, Endpoint: {c.endpoint_url}"
    )

# Filter for Azure OpenAI connections
aoai_conns = project_client.connections.list(
    connection_type=ConnectionType.AZURE_OPEN_AI
)
print(f"\n🌀 Found {len(aoai_conns)} Azure OpenAI connections:")
for c in aoai_conns:
    print(f"   -> {c.name}")

# Get default connection of type AZURE_AI_SERVICES
default_conn = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SERVICES, include_credentials=False
)
if default_conn:
    print("\n⭐ Default Azure AI Services connection:")
    print(default_conn)
else:
    print("No default connection found for Azure AI Services.")

🔎 Found 6 total connections.
1) Name: hub-demo-ivii-connection-AISearch, Type: ConnectionType.AZURE_AI_SEARCH, Endpoint: https://agent-ai-search-ivii.search.windows.net
2) Name: hub-demo-ivii-connection-AIServices_aoai, Type: ConnectionType.AZURE_OPEN_AI, Endpoint: https://agent-ai-servicesivii.openai.azure.com
3) Name: hub-demo-ivii-connection-AIServices, Type: ConnectionType.AZURE_AI_SERVICES, Endpoint: https://agent-ai-servicesivii.cognitiveservices.azure.com
4) Name: hub-demo-ivii-connection-BingSearch, Type: ConnectionType.API_KEY, Endpoint: https://api.bing.microsoft.com
5) Name: project-demo-ivii/workspaceblobstore, Type: ConnectionType.AZURE_BLOB_STORAGE, Endpoint: https://agentstorageivii.core.windows.net/21203267-ab62-41f0-bc84-12a9170fba7b-azureml-blobstore
6) Name: project-demo-ivii/workspaceartifactstore, Type: ConnectionType.AZURE_BLOB_STORAGE, Endpoint: https://agentstorageivii.core.windows.net/21203267-ab62-41f0-bc84-12a9170fba7b-azureml

🌀 Found 1 Azure OpenAI connecti

# 4. Observabilidad y Trazabilidad

Queremos **collect telemetry** de nuestras llamadas a LLM, por ejemplo:
- Marcas de tiempo de las solicitudes.
- Latencia.
- Errores potenciales.
- Opcionalmente, las solicitudes y respuestas reales (si activas el registro de contenido).

Con esta información, podremos diagnosticar problemas y optimizar el rendimiento de nuestros modelos.

Mostraremos cómo configurar:
1. **Azure Monitor** instrumentation with Application Insights.
2. **Viewing** your traces in Azure AI Foundry's portal.

### 4.1 Habilitar OpenTelemetry para Azure AI Inference
  
Establecemos variables de entorno para asegurar:
1. **Prompt content** se captura (optional!)
2. El **Azure SDK** utiliza OpenTelemetry como implementación de trazabilidad.
3. Llamamos a `AIInferenceInstrumentor().instrument()` para parchear y habilitar la instrumentación.


In [4]:
import os
from azure.ai.inference.tracing import AIInferenceInstrumentor

# (Optional) capture prompt & completion contents in traces
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"  # or 'false'

# Let the Azure SDK know we want to use OpenTelemetry
os.environ["AZURE_SDK_TRACING_IMPLEMENTATION"] = "opentelemetry"

# Instrument the Azure AI Inference client library
AIInferenceInstrumentor().instrument()
print("✅ Azure AI Inference instrumentation enabled.")

✅ Azure AI Inference instrumentation enabled.


Ahora configuraremos el tracing a **Application Insights**, que reenviará tus logs a la página de **Azure AI Foundry** **Tracing**.

**Steps**:
1. En AI Foundry, ve a la pestaña **Tracing** de tu proyecto, y asocia (o crea) un recurso de **Application Insights**.
2. En el código, llama a `project_client.telemetry.get_connection_string()` para obtener la clave de instrumentación.
3. Usa `azure.monitor.opentelemetry.configure_azure_monitor(...)` con esa conexión.
4. Realiza una llamada de inferencia -> los logs aparecerán en el portal de Foundry (y en Azure Monitor).


In [5]:
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.ai.inference.models import UserMessage

app_insights_conn_str = project_client.telemetry.get_connection_string()
if app_insights_conn_str:
    print("🔧 Found App Insights connection string, configuring...")
    configure_azure_monitor(connection_string=app_insights_conn_str)
    # Optionally add more instrumentation (for openai or langchain):
    project_client.telemetry.enable()

    # Let's do a test call that logs to AI Foundry's Tracing page
    try:
        with project_client.inference.get_chat_completions_client() as client:
            prompt_msg = "Any easy at-home cardio exercise recommendations?"
            response = client.complete(
                model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
                messages=[UserMessage(content=prompt_msg)],
            )
            print("\n🤖 Response (logged to App Insights):")
            print(response.choices[0].message.content)
    except Exception as e:
        print("❌ Chat completions with Azure Monitor example failed:", e)
else:
    print("No Application Insights connection string is configured in this project.")

🔧 Found App Insights connection string, configuring...

🤖 Response (logged to App Insights):
Absolutely! Here are some easy at-home cardio exercises to get your heart rate up and improve your cardiovascular fitness:

1. **Jumping Jacks**: A great full-body workout that also improves coordination.

2. **High Knees**: Run in place while lifting your knees as high as possible. This is excellent for the lower body and core.

3. **Burpees**: A bit more intense, but they combine a squat, push-up, and jump for a full-body workout.

4. **Mountain Climbers**: Start in a plank position and alternate bringing your knees towards your chest. This works your core and shoulders.

5. **Skipping rope**: Simple, effective, and fun. It requires a jump rope but provides an excellent cardio workout.

6. **Dancing**: Put on your favorite music and dance around your living room. It’s fun and a great way to get your heart rate up.

7. **Shadow Boxing**: No equipment needed. Just throw punches in the air and i

### 4.3 Visualización de Trazas en Azure AI Foundry
Después de ejecutar el código anterior:
1. Ve a tu proyecto de AI Foundry.
2. Haz clic en **Tracing** en la barra lateral.
3. Deberías ver los registros (logs) de tus llamadas.
4. Filtra, expande o explora los detalles según lo necesites.

Además, si deseas paneles de control más avanzados, puedes abrir tu recurso de **Application Insights** desde Foundry. En el portal de App Insights obtendrás otras funciones como detalles de transacciones de extremo a extremo, registros de consultas, etc.



# 5. Agent-based Example

Ahora vamos a crear un **Health Resource Agent** que hace referencia a documentos de ejemplo sobre recetas o pautas, y luego demostramos:
1. La creación de un Agente con instrucciones.
2. La creación de un hilo de conversación.
3. La ejecución de consultas en múltiples pasos con **observability** habilitada.
4. Opcionalmente, la limpieza de recursos al final.

> El enfoque basado en agentes es útil cuando deseas flujos de conversación más sofisticados o **tool usage** (como la búsqueda en archivos).


## 5.1 Create Sample Files & Vector Store
We'll create dummy `.md` files about recipes/guidelines, then push them into a **vector store** so our agent can do semantic search.

(*This portion is a quick summary—see [the other file-search tutorial] if you need more details.)

In [6]:
from azure.ai.projects.models import (
    FileSearchTool,
    FilePurpose,
    MessageTextContent,
    MessageRole,
)


def create_sample_files():
    """Create some local .md files with sample text."""
    recipes_md = """# Base de Datos de Recetas Saludables\n\n"
        "## Recetas sin Gluten\n"
        "1. Tazón de Quinoa\n"
        "   - Ingredientes: quinoa, verduras, aceite de oliva\n"
        "   - Instrucciones: Cocer la quinoa, agregar verduras\n\n"
        "2. Pasta de Arroz\n"
        "   - Ingredientes: pasta de arroz, verduras mixtas\n"
        "   - Instrucciones: Hervir la pasta, saltear las verduras\n\n"
        "## Recetas para Diabéticos\n"
        "1. Salteado Bajo en Carbohidratos\n"
        "   - Ingredientes: pollo, verduras, salsa de tamari\n"
        "   - Instrucciones: Cocinar el pollo, agregar verduras\n\n"
        "## Recetas Saludables para el Corazón\n"
        "1. Salmón al Horno\n"
        "   - Ingredientes: salmón, limón, hierbas\n"
        "   - Instrucciones: Sazonar el salmón, hornear\n\n"
        "2. Tazón Mediterráneo\n"
        "   - Ingredientes: garbanzos, verduras, tahini\n"
        "   - Instrucciones: Combinar ingredientes\n"""

    guidelines_md = """# Pautas Alimentarias

        ## Pautas Generales
        - Consume una variedad de alimentos
        - Controla el tamaño de las porciones
        - Mantente hidratado

        ## Dietas Especiales
        1. Dieta sin Gluten
        - Evita el trigo, la cebada y el centeno
        - Enfócate en alimentos naturalmente sin gluten

        2. Dieta para Diabéticos
        - Monitorea la ingesta de carbohidratos
        - Elige alimentos de bajo índice glucémico

        3. Dieta Saludable para el Corazón
        - Limita las grasas saturadas
        - Elige proteínas magras"""

    with open("recipes.md", "w", encoding="utf-8") as f:
        f.write(recipes_md)
    with open("guidelines.md", "w", encoding="utf-8") as f:
        f.write(guidelines_md)

    print("📄 Created sample resource files: recipes.md, guidelines.md")
    return ["recipes.md", "guidelines.md"]


sample_files = create_sample_files()


def create_vector_store(files, store_name="my_health_resources"):
    try:
        uploaded_ids = []
        for fp in files:
            upl = project_client.agents.upload_file_and_poll(
                file_path=fp, purpose=FilePurpose.AGENTS  # Add FilePurpose.AGENTS here
            )
            uploaded_ids.append(upl.id)
            print(f"✅ Uploaded: {fp} -> File ID: {upl.id}")

        # Create vector store from these file IDs
        vs = project_client.agents.create_vector_store_and_poll(
            file_ids=uploaded_ids, name=store_name
        )
        print(f"🎉 Created vector store '{store_name}', ID: {vs.id}")
        return vs, uploaded_ids
    except Exception as e:
        print(f"❌ Error creating vector store: {e}")
        return None, []


vector_store, file_ids = None, []
if sample_files:
    vector_store, file_ids = create_vector_store(
        sample_files, store_name="health_resources_example"
    )

📄 Created sample resource files: recipes.md, guidelines.md
✅ Uploaded: recipes.md -> File ID: assistant-4FUBeBupfHfJRPTXerDZWn
✅ Uploaded: guidelines.md -> File ID: assistant-F84Yj13sDtAuVgNCCTkmMy
❌ Error creating vector store: (None) Failed to create vector store.
Code: None
Message: Failed to create vector store.


## 5.2 Create a Health Resource Agent

Crearemos un **FileSearchTool** que haga referencia al vector store, luego crearemos un agente con instrucciones de que debe:

1. Proporcionar descargos de responsabilidad.
2. Ofrecer consejos generales sobre nutrición o recetas.
3. Citar fuentes cuando sea posible.
4. Fomentar la consulta con profesionales para obtener asesoramiento médico más completo.




In [7]:
from azure.ai.projects.models import FileSearchTool, FilePurpose
from azure.ai.projects.models import ConnectionType, MessageTextContent, MessageRole


def create_health_agent(vs_id):
    try:
        # The tool references our vector store so the agent can search it
        file_search_tool = FileSearchTool(vector_store_ids=[vs_id])

        instructions = """
            Eres un asesor de recursos de salud con acceso a archivos de recetas y guías dietéticas.
            Tú:
            1. Siempre presenta exenciones de responsabilidad (no eres un profesional médico)
            2. Proporciona referencias a archivos cuando sea posible.
            3. Enfócate en consejos generales de nutrición o recetas.
            4. Fomenta la consulta con profesionales para obtener un asesoramiento más detallado.
        """

        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
            name="health-search-agent",
            instructions=instructions,
            tools=file_search_tool.definitions,
            tool_resources=file_search_tool.resources,
        )
        print(f"🎉 Created agent '{agent.name}' with ID: {agent.id}")
        return agent
    except Exception as e:
        print(f"❌ Error creating health agent: {e}")
        return None


health_agent = None
if vector_store:
    health_agent = create_health_agent(vector_store.id)

## 5.3 Using the Agent
Vamos a crear un n**thread** y preguntarle al agente algunas preguntas. Nos apoyaremos en la configuración de **observability** que ya establecimos para que cada paso se rastree.


In [8]:
def create_thread():
    try:
        thread = project_client.agents.create_thread()
        print(f"📝 Created new thread, ID: {thread.id}")
        return thread
    except Exception as e:
        print(f"❌ Could not create thread: {e}")
        return None


def ask_question(thread_id, agent_id, user_question):
    try:
        # 1) Add user message
        msg = project_client.agents.create_message(
            thread_id=thread_id, role="user", content=user_question
        )
        print(f"User asked: '{user_question}'")
        # 2) Create & process a run
        run = project_client.agents.create_and_process_run(
            thread_id=thread_id, agent_id=agent_id
        )
        print(f"Run finished with status: {run.status}")
        if run.last_error:
            print("Error details:", run.last_error)
        return run
    except Exception as e:
        print(f"❌ Error asking question: {e}")
        return None


if health_agent:
    thread = create_thread()
    if thread:
        # Let's ask a few sample questions
        queries = [
            "¿Podrías sugerir una receta de almuerzo sin gluten?",
            "Muéstrame algunas ideas de comidas saludables para el corazón.",
            "¿Qué pautas tienes para alguien con diabetes?",
        ]
        for q in queries:
            ask_question(thread.id, health_agent.id, q)

### 5.3.1 Viewing the conversation
We can retrieve the conversation messages to see how the agent responded, check if it cited file passages, etc.

In [9]:
def display_thread(thread_id):
    try:
        messages = project_client.agents.list_messages(thread_id=thread_id)
        print("\n🗣️ Conversation:")
        for m in reversed(messages.data):
            if m.content:
                last_content = m.content[-1]
                if hasattr(last_content, "text"):
                    print(f"[{m.role.upper()}]: {last_content.text.value}\n")

        print("\n📎 Checking for citations...")
        for c in messages.file_citation_annotations:
            print(
                f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}"
            )
    except Exception as e:
        print(f"❌ Could not display thread: {e}")


# If we created a thread above, let's read it
if health_agent and thread:
    display_thread(thread.id)

# 6. Cleanup
If desired, we can remove the vector store, files, and agent to keep things tidy. (In a real solution, you might keep them around.)

In [14]:
def cleanup_resources():
    try:
        if "vector_store" in globals() and vector_store:
            project_client.agents.delete_vector_store(vector_store.id)
            print("🗑️ Deleted vector store.")

        if "file_ids" in globals() and file_ids:
            for fid in file_ids:
                project_client.agents.delete_file(fid)
            print("🗑️ Deleted uploaded files.")

        if "health_agent" in globals() and health_agent:
            project_client.agents.delete_agent(health_agent.id)
            print("🗑️ Deleted health agent.")

        if "sample_files" in globals() and sample_files:
            for sf in sample_files:
                if os.path.exists(sf):
                    os.remove(sf)
            print("🗑️ Deleted local sample files.")
    except Exception as e:
        print(f"❌ Error cleaning up: {e}")


cleanup_resources()

🗑️ Deleted vector store.
🗑️ Deleted uploaded files.
🗑️ Deleted health agent.
🗑️ Deleted local sample files.
